In [3]:
import pandas as pd
import oracledb
import os

# ==========================================
# 1. 設定區域 (路徑修正版)
# ==========================================
# 根據您的下載路徑，通常解壓縮後會多一層 "instantclient_19_29"
# 請確認這個路徑下真的有 oci.dll
CLIENT_DIR = r"C:\Users\lin09\Desktop\Lab\instantclient-basic-windows.x64-19.29.0.0.0dbru\instantclient_19_29"

DB_USER = "MIC_2026"
DB_PASS = "2026"
DB_HOST = "140.113.59.168"
DB_PORT = "1521"
DB_SID  = "xe"

# 目標 HID
TARGET_HID = 'fd9f8472-099f-166e-c837-416d8bd1be3e'
# ==========================================

def query_specific_hid():
    print(f"🔌 嘗試連接資料庫 (Thick Mode)...")
    print(f"📂 指定驅動路徑: {CLIENT_DIR}")
    
    try:
        # 1. 初始化 Oracle Client
        try:
            oracledb.init_oracle_client(lib_dir=CLIENT_DIR)
        except Exception as e:
            # 如果已經初始化過，這裡會報錯，我們可以忽略
            print(f"ℹ️ Client 初始化訊息: {e}")

        # 2. 建立連線
        dsn_tns = oracledb.makedsn(DB_HOST, DB_PORT, sid=DB_SID)
        connection = oracledb.connect(user=DB_USER, password=DB_PASS, dsn=dsn_tns)
        print("✅ 連線成功！")
        
        # 3. 執行 SQL 查詢
        print(f"🔍 正在搜尋 HID: {TARGET_HID} ...")
        sql = f"""
        SELECT *
        FROM CT_SELF
        WHERE HID = '{TARGET_HID}'
        ORDER BY TIMESTAMP ASC
        """
        
        df = pd.read_sql(sql, connection)
        connection.close()
        
        if len(df) == 0:
            print("❌ 查無資料 (但連線是成功的)")
            return

        print("\n" + "="*80)
        print(f"📄 HID 履歷表 (共 {len(df)} 筆紀錄)")
        print("="*80)
        
        # 顯示設定
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        
        # 挑選顯示欄位
        cols = ['SOURCE', 'TIMESTAMP', 'TIMEDIFF', 'STAGE_NAME', 'CALTYPE']
        print(df[[c for c in cols if c in df.columns]])
        print("="*80)

    except Exception as e:
        print(f"\n❌ 發生錯誤: {e}")
        print("💡 故障排除建議：")
        print("1. 請確認 CLIENT_DIR 路徑裡真的有 oci.dll 檔案。")
        print("2. 如果路徑裡只有一堆資料夾，請再點進去一層看看。")

if __name__ == "__main__":
    query_specific_hid()

🔌 嘗試連接資料庫 (Thick Mode)...
📂 指定驅動路徑: C:\Users\lin09\Desktop\Lab\instantclient-basic-windows.x64-19.29.0.0.0dbru\instantclient_19_29
✅ 連線成功！
🔍 正在搜尋 HID: fd9f8472-099f-166e-c837-416d8bd1be3e ...

📄 HID 履歷表 (共 13 筆紀錄)
                                               SOURCE                        TIMESTAMP    TIMEDIFF        STAGE_NAME                            CALTYPE
0     GKG.GT++-N.GT936MV.C10278.H2-2F-SMT-04H-PRINTER  05-1月 -26 08.40.12.111807000 上午   74.471393              None            workstarted-workstarted
1   KOHYOUNG.KY8030-2.SS-SX-00369.D10014.H2-2F-SMT...  05-1月 -26 08.41.22.850954000 上午   21.820266              None            workstarted-workstarted
2   FUJI.M6III.SE0170966-67.C10270.H2-2F-SMT-04H-M...  05-1月 -26 08.42.48.503452000 上午   26.220096  Logical Module 1  workstagestarted-workstagestarted
3   FUJI.M6III.SE0170966-67.C10270.H2-2F-SMT-04H-M...  05-1月 -26 08.42.59.524109000 上午   22.601626  Logical Module 2  workstagestarted-workstagestarted
4   FUJI.M6III.SE0170966-6

C:\Users\lin09\AppData\Local\Temp\ipykernel_592\1684447195.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, connection)
